# Agno + Strata Integration

This tutorial demonstrates how to build AI agents using an Agno with Klavis Strata MCP servers for enhanced functionality.

## Prerequisites

Before we begin, you'll need:

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)

In [ ]:
# Install the required packages
%pip install -q klavis openai agno

In [ ]:
import os
import webbrowser
import asyncio
from klavis import Klavis
from klavis.types import McpServerName, ToolFormat
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.mcp import MCPTools


# Set environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual OpenAI API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key

## Step 1: Create Strata MCP Server

Create a unified MCP server that combines multiple services (Gmail and Slack) for enhanced agent capabilities.

In [ ]:
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

# Create a Strata MCP server with Gmail and Slack integrations
response = klavis_client.mcp_server.create_strata_server(
    user_id="1234",
    servers=[McpServerName.GMAIL, McpServerName.SLACK],
)

print(f"🚀 Strata MCP server created successfully!")

# Handle OAuth authorization if needed
if response.oauth_urls:
    for server_name, oauth_url in response.oauth_urls.items():
        webbrowser.open(oauth_url)
        print(f"🔐 Opening OAuth authorization for {server_name}")
        input(f"Press Enter after completing {server_name} OAuth authorization...")

## Step 2: Create Agent Loop

Implement an agentic loop that allows Agent to use MCP tools to complete tasks.

In [ ]:
async def agno_with_mcp_server(mcp_server_url: str, user_query: str):
    """Run an Agno agent with Klavis MCP server tools."""

    async with MCPTools(transport="streamable-http", url=mcp_server_url) as mcp_tools:
        agent = Agent(
            model=OpenAIChat(
                id="gpt-4o",
                api_key=os.getenv("OPENAI_API_KEY")
            ),
            instructions="You are a helpful AI assistant.",
            tools=[mcp_tools],
            markdown=True,
        )

        response = await agent.arun(user_query)
        return response.content

## Step 3: Run the  Agent

In [ ]:
async def main():
    result = await agno_with_mcp_server(
        mcp_server_url=response.strata_server_url,
        user_query="Check my latest 5 emails and summarize them in a Slack message to #general"
    )
    print(f"\nFinal Response: {result}")


if __name__ == "__main__":
    asyncio.run(main())

## Summary

🎉 Congratulations! You've successfully created an Agno agent that can:

1. **Read emails** using the Gmail MCP server
2. **Send Slack messages** using the Slack MCP server
3. **Coordinate multiple services** through Klavis Strata MCP integration

This demonstrates the power of combining Agno's advanced reasoning capabilities with Klavis MCP servers for building sophisticated AI workflows that can interact with multiple external services seamlessly.